In [64]:
from odps import ODPS
from odps import options
import oss2
from odps.df import DataFrame
import datetime
import pandas as pd
import numpy as np
import threading
import queue

# connect信息
from ACCESS import ACCESS_ID
from ACCESS import SECRET_ACCESS_KEY
from ACCESS import ODPS_PROJECT
from ACCESS import ODPS_ENDPOINT
#FILE_PATH = ''

# # 此为外网地址
OSS_ENDPOINT = 'http://oss-cn-shanghai.aliyuncs.com'
OSS_BUCKET = 'hypertrons'

# oss实例
auth = oss2.Auth(ACCESS_ID, SECRET_ACCESS_KEY)
bucket = oss2.Bucket(auth, OSS_ENDPOINT, OSS_BUCKET)

# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY, project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

# 总时间,单位天
total_date = 180

# 时间间隔,7天
interval = 7

times = int(total_date / interval)

# 当前时间
now = datetime.datetime.now()
delta = datetime.timedelta(days=total_date)

# 开始时间
begin_at = now - delta
begin_at_str = begin_at.strftime('%Y-%m-%d %H:%M:%S')

# 权重
ISSUECOMMENT_WEIGHT = 1
OPENISSUE_WEIGHT = 2
OPENPULL_WEIGHT = 3
PULLREVIEW_WEIGHT = 4
MERGEPULL_WEIGHT = 2
WATCH_WEIGHT = 1
FORK_WEIGHT = 2

TEMP_TABLE_NAME = 'TEMP_TABLE_NAME'

# 创建临时数据库
drop_sql = '''
DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
(
    actor_login STRING
);
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220427130441568gny8n4pr2>

In [59]:
# 分区
PT = '20210601'

for i in range(times + 1):
    add_date = (i + 1) * interval
    print(add_date)
    end_at_str = ''
    if i != times:
        delta = datetime.timedelta(days=add_date)
        n_days = begin_at + delta
        end_at_str = n_days.strftime('%Y-%m-%d %H:%M:%S')
    else:
        end_at_str = now.strftime('%Y-%m-%d %H:%M:%S')
    print(begin_at_str)
    print(end_at_str)
    select_sql = '''
    set odps.sql.validate.orderby.limite=false;
    INSERT INTO {TEMP_TABLE_NAME}
    SELECT actor_id FROM ods_github_log_year2021_d where PT = '20210601'
    '''.format(
        TEMP_TABLE_NAME=TEMP_TABLE_NAME
    )
    table = o.execute_sql(select_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

7
2021-10-29 19:52:59
2021-11-05 19:52:59
14
2021-10-29 19:52:59
2021-11-12 19:52:59
21
2021-10-29 19:52:59
2021-11-19 19:52:59
28
2021-10-29 19:52:59
2021-11-26 19:52:59
35
2021-10-29 19:52:59
2021-12-03 19:52:59
42
2021-10-29 19:52:59
2021-12-10 19:52:59
49
2021-10-29 19:52:59
2021-12-17 19:52:59
56
2021-10-29 19:52:59
2021-12-24 19:52:59
63
2021-10-29 19:52:59
2021-12-31 19:52:59
70
2021-10-29 19:52:59
2022-01-07 19:52:59
77
2021-10-29 19:52:59
2022-01-14 19:52:59
84
2021-10-29 19:52:59
2022-01-21 19:52:59
91
2021-10-29 19:52:59
2022-01-28 19:52:59
98
2021-10-29 19:52:59
2022-02-04 19:52:59
105
2021-10-29 19:52:59
2022-02-11 19:52:59
112
2021-10-29 19:52:59
2022-02-18 19:52:59
119
2021-10-29 19:52:59
2022-02-25 19:52:59
126
2021-10-29 19:52:59
2022-03-04 19:52:59
133
2021-10-29 19:52:59
2022-03-11 19:52:59
140
2021-10-29 19:52:59
2022-03-18 19:52:59
147
2021-10-29 19:52:59
2022-03-25 19:52:59
154
2021-10-29 19:52:59
2022-04-01 19:52:59
161
2021-10-29 19:52:59
2022-04-08 19:52:59
168

In [63]:
end_at_str = now.strftime('%Y-%m-%d %H:%M:%S')
# 获取需要导出的项目
select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT actor_id FROM {TEMP_TABLE_NAME}
    '''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME,
    begin_at_str=begin_at_str,
    end_at_str=end_at_str
)
table = o.execute_sql(select_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})
print('获取数据成功')

获取数据成功


In [20]:
exitFlag = 0
# 定义多线程
class putOssThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    def run(self):
        process_data(self.name, self.q);

def process_data(file_name, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            q_data = q.get()
            print(q_data)
            bucket.put_object(q_data['file_name'], q_data['oss_csv'])
            queueLock.release()
        else:
            queueLock.release()

threadList = ['Thread-1', 'Thread-2', 'Thread-3']
queueLock = threading.Lock()
workQueue = queue.Queue(10)
threads = []
threadID = 1
print('创建新线程')
# 创建新线程
for tName in threadList:
    thread = putOssThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

with table.open_reader(tunnel=True) as reader:
    oss_rs = pd.DataFrame(columns=('id', 'date', 'value'))
    cur_repo = ''
    cur_repo_name = ''
    for record in reader:
        if record.repo_id != cur_repo:  # 若换了repo_id，则写入oss 清空oss_rs
            print(cur_repo)
            print(oss_rs)
            if cur_repo != '':
                file_name = FILE_PATH + now.strftime(‘%Y-%m-%d’) + ‘/’ + cur_repo_name + ‘.csv’
                q_data = {}
                q_data['file_name']= file_name
                q_data['oss_csv'] = http://oss_rs.to_csv(index=False)
                # 填充队列
                queueLock.acquire()
                workQueue.put(q_data)
                queueLock.release()
                oss_rs = pd.DataFrame(columns=('id', 'date', 'value'))

            cur_repo = record.repo_id
            cur_repo_name = record.repo_name
        else:  # 没换则添加oss_rs缓存
            oss_rs = oss_rs.append([{'id': record.actor_login, 'date': record.end_date, 'value': record.score}], ignore_index=True)

file_name = FILE_PATH + now.strftime('%Y-%m-%d') + '/' + record.repo_name + '.csv'
bucket.put_object(file_name, http://oss_rs.to_csv(index=False))

# 等待队列清空
while not workQueue.empty():
    pass

# 通知线程是时候退出
exitFlag = 1

# 等待所有线程完成
for t in threads:
    t.join()

print ('退出主线程')

SyntaxError: invalid character in identifier (<ipython-input-20-340faba96132>, line 11)